# Frame Clarity Check

### OpenCV Laplacian Variance

In [18]:
import cv2
import os
import re

def is_blurry(image_path, threshold=100):
    """Check if an image is blurry using the Laplacian variance method."""
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
    laplacian_var = cv2.Laplacian(image, cv2.CV_64F).var()  # Compute variance
    return laplacian_var < threshold, laplacian_var  # Returns (is_blurry, score)

def extract_numbers(filename):
    """Extract the numeric parts from the filename to sort properly."""
    # Use regular expressions to extract two numbers from the filename like 'frame_1_6.jpg'
    match = re.match(r'frame_(\d+)_(\d+)', filename)
    if match:
        return int(match.group(1)), int(match.group(2))  # Return as a tuple (first_number, second_number)
    return 0, 0  # Default if match fails

# Path to frames folder
frames_folder = "frames/video_2025-02-22_00-39-13"  # Change to your folder

# List to store blurry images
blurry_images = []

# Check each frame (sorted sequentially by numeric part of filename)
for filename in sorted(os.listdir(frames_folder), key=extract_numbers):
    image_path = os.path.join(frames_folder, filename)
    
    if filename.endswith(".jpg"):
        blurry, score = is_blurry(image_path)
        print(f"{filename} - Sharpness Score: {score} {'(Blurry)' if blurry else '(Clear)'}")
        
        if blurry:
            blurry_images.append(filename)  # Store blurry image names
            # os.remove(image_path)  # Delete blurry image
            # print(f"Deleted: {filename}")

# Print all blurry images in sequential order
if blurry_images:
    print("\nBlurry Images Detected & Deleted:")
    for img in blurry_images:
        print(img)
else:
    print("\nNo blurry images detected.")

print("Blurry image check completed.")

frame_1_1.jpg - Sharpness Score: 163.56591791072418 (Clear)
frame_1_2.jpg - Sharpness Score: 172.3000650293986 (Clear)
frame_1_3.jpg - Sharpness Score: 170.91814118829305 (Clear)
frame_1_4.jpg - Sharpness Score: 171.55101551903613 (Clear)
frame_1_5.jpg - Sharpness Score: 179.76890237934325 (Clear)
frame_1_6.jpg - Sharpness Score: 179.2472454420302 (Clear)
frame_1_7.jpg - Sharpness Score: 200.56465776645874 (Clear)
frame_1_8.jpg - Sharpness Score: 184.8912532183223 (Clear)
frame_1_9.jpg - Sharpness Score: 170.47196281406607 (Clear)
frame_1_10.jpg - Sharpness Score: 181.87460892533196 (Clear)
frame_2_1.jpg - Sharpness Score: 136.85128672175935 (Clear)
frame_2_2.jpg - Sharpness Score: 157.4436528103193 (Clear)
frame_2_3.jpg - Sharpness Score: 185.0310420234574 (Clear)
frame_2_4.jpg - Sharpness Score: 157.24384992725584 (Clear)
frame_2_5.jpg - Sharpness Score: 140.62313566012915 (Clear)
frame_2_6.jpg - Sharpness Score: 148.7392119405111 (Clear)
frame_2_7.jpg - Sharpness Score: 156.22518505

### FSRCNN (Fast Super-Resolution Convolutional Neural Network)

In [19]:
import cv2
import os
import re
import numpy as np

# def apply_superres(image):
#     """Use OpenCV's DNN super-resolution model to estimate image sharpness."""
#     # Load pre-trained DNN model for super-resolution
#     sr = cv2.dnn_superres.DnnSuperResImpl_create()
#     model_path = "models/EDSR_x3.pb"  # Path to pre-trained model, such as EDSR or any other
#     sr.readModel(model_path)
    
#     # Set the desired scale factor (e.g., 3x)
#     sr.setModel("edsr", 3)
    
#     # Apply the super-resolution model
#     result = sr.upsample(image)
#     return result

def apply_superres(image):
    """Use OpenCV's DNN super-resolution model to estimate image sharpness."""
    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    model_path = "models/FSRCNN_x4.pb"  # Replace with FSRCNN model path
    sr.readModel(model_path)
    sr.setModel("fsrcnn", 4)  # Set model type to "fsrcnn"
    result = sr.upsample(image)
    return result


def is_blurry(image_path, threshold=7):
    """Check if an image is blurry using Laplacian variance and super-res model."""
    image = cv2.imread(image_path)

    # Apply the super-resolution model to the image
    enhanced_image = apply_superres(image)
    
    # Calculate Laplacian variance for sharpness of the enhanced image
    gray_enhanced = cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2GRAY)
    laplacian_var = cv2.Laplacian(gray_enhanced, cv2.CV_64F).var()

    # If the model doesn't improve the sharpness, it's blurry
    return laplacian_var < threshold, laplacian_var

def extract_numbers(filename):
    """Extract the numeric parts from the filename to sort properly."""
    match = re.match(r'frame_(\d+)_(\d+)', filename)
    if match:
        return int(match.group(1)), int(match.group(2))
    return 0, 0

# Path to frames folder
frames_folder = "frames/video_2025-02-22_00-39-13"  # Change to your folder

# List to store blurry images
blurry_images = []

# Check each frame (sorted sequentially by numeric part of filename)
for filename in sorted(os.listdir(frames_folder), key=extract_numbers):
    image_path = os.path.join(frames_folder, filename)
    
    if filename.endswith(".jpg"):
        blurry, score = is_blurry(image_path)
        print(f"{filename} - Sharpness Score: {score} {'(Blurry)' if blurry else '(Clear)'}")
        
        if blurry:
            blurry_images.append(filename)  # Store blurry image names
            # os.remove(image_path)  # Delete blurry image
            print(f"Deleted: {filename}")

# Print all blurry images in sequential order
if blurry_images:
    print("\nBlurry Images Detected & Deleted:")
    for img in blurry_images:
        print(img)
else:
    print("\nNo blurry images detected.")

print("Blurry image check completed.")

frame_1_1.jpg - Sharpness Score: 8.136494839134775 (Clear)
frame_1_2.jpg - Sharpness Score: 8.34047676019665 (Clear)
frame_1_3.jpg - Sharpness Score: 8.33300533071057 (Clear)
frame_1_4.jpg - Sharpness Score: 8.372113405595185 (Clear)
frame_1_5.jpg - Sharpness Score: 8.49724240756592 (Clear)
frame_1_6.jpg - Sharpness Score: 8.484291109467513 (Clear)
frame_1_7.jpg - Sharpness Score: 9.06336347816718 (Clear)
frame_1_8.jpg - Sharpness Score: 8.617567490003132 (Clear)
frame_1_9.jpg - Sharpness Score: 8.33705325557848 (Clear)
frame_1_10.jpg - Sharpness Score: 8.543933779549434 (Clear)
frame_2_1.jpg - Sharpness Score: 7.663695573435723 (Clear)
frame_2_2.jpg - Sharpness Score: 8.015623692828372 (Clear)
frame_2_3.jpg - Sharpness Score: 9.333650772328133 (Clear)
frame_2_4.jpg - Sharpness Score: 8.564647717974207 (Clear)
frame_2_5.jpg - Sharpness Score: 7.738506948150528 (Clear)
frame_2_6.jpg - Sharpness Score: 7.993258191508551 (Clear)
frame_2_7.jpg - Sharpness Score: 8.432413180080383 (Clear)
f

In [20]:
import cv2
import os
import re
import numpy as np

def apply_superres(image):
    """Use OpenCV's DNN super-resolution model to estimate image sharpness."""
    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    model_path = "models/FSRCNN_x3.pb"  # Replace with FSRCNN model path
    sr.readModel(model_path)
    sr.setModel("fsrcnn", 3)  # Set model type to "fsrcnn"
    result = sr.upsample(image)
    return result


def is_blurry(image_path, threshold=10):
    """Check if an image is blurry using Laplacian variance and super-res model."""
    image = cv2.imread(image_path)

    # Apply the super-resolution model to the image
    enhanced_image = apply_superres(image)
    
    # Calculate Laplacian variance for sharpness of the enhanced image
    gray_enhanced = cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2GRAY)
    laplacian_var = cv2.Laplacian(gray_enhanced, cv2.CV_64F).var()

    # If the model doesn't improve the sharpness, it's blurry
    return laplacian_var < threshold, laplacian_var

def extract_numbers(filename):
    """Extract the numeric parts from the filename to sort properly."""
    match = re.match(r'frame_(\d+)_(\d+)', filename)
    if match:
        return int(match.group(1)), int(match.group(2))
    return 0, 0

# Path to frames folder
frames_folder = "frames/video_2025-02-22_00-39-13"  # Change to your folder

# List to store blurry images
blurry_images = []

# Check each frame (sorted sequentially by numeric part of filename)
for filename in sorted(os.listdir(frames_folder), key=extract_numbers):
    image_path = os.path.join(frames_folder, filename)
    
    if filename.endswith(".jpg"):
        blurry, score = is_blurry(image_path)
        print(f"{filename} - Sharpness Score: {score} {'(Blurry)' if blurry else '(Clear)'}")
        
        if blurry:
            blurry_images.append(filename)  # Store blurry image names
            # os.remove(image_path)  # Delete blurry image
            print(f"Deleted: {filename}")

# Print all blurry images in sequential order
if blurry_images:
    print("\nBlurry Images Detected & Deleted:")
    for img in blurry_images:
        print(img)
else:
    print("\nNo blurry images detected.")

print("Blurry image check completed.")

frame_1_1.jpg - Sharpness Score: 12.349059763548112 (Clear)
frame_1_2.jpg - Sharpness Score: 12.727307455222459 (Clear)
frame_1_3.jpg - Sharpness Score: 12.639508923561182 (Clear)
frame_1_4.jpg - Sharpness Score: 12.650918907782476 (Clear)
frame_1_5.jpg - Sharpness Score: 13.060873844783963 (Clear)
frame_1_6.jpg - Sharpness Score: 13.02486386590533 (Clear)
frame_1_7.jpg - Sharpness Score: 14.12983636162945 (Clear)
frame_1_8.jpg - Sharpness Score: 13.265414018037573 (Clear)
frame_1_9.jpg - Sharpness Score: 12.621130615469722 (Clear)
frame_1_10.jpg - Sharpness Score: 13.068183519902911 (Clear)
frame_2_1.jpg - Sharpness Score: 11.498650756810294 (Clear)
frame_2_2.jpg - Sharpness Score: 12.212452747778118 (Clear)
frame_2_3.jpg - Sharpness Score: 14.139029983990296 (Clear)
frame_2_4.jpg - Sharpness Score: 12.61843217535437 (Clear)
frame_2_5.jpg - Sharpness Score: 11.624723849260063 (Clear)
frame_2_6.jpg - Sharpness Score: 12.056392896235694 (Clear)
frame_2_7.jpg - Sharpness Score: 12.597826

In [21]:
import cv2
import os
import re
import numpy as np

def apply_superres(image):
    """Use OpenCV's DNN super-resolution model to estimate image sharpness."""
    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    model_path = "models/FSRCNN_x3.pb"  # Replace with FSRCNN model path
    sr.readModel(model_path)
    sr.setModel("fsrcnn", 3)  # Set model type to "fsrcnn"
    result = sr.upsample(image)
    return result


def is_blurry(image_path, threshold=7):
    """Check if an image is blurry using Laplacian variance and super-res model."""
    image = cv2.imread(image_path)

    # Apply the super-resolution model to the image
    enhanced_image = apply_superres(image)
    
    # Calculate Laplacian variance for sharpness of the enhanced image
    gray_enhanced = cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2GRAY)
    laplacian_var = cv2.Laplacian(gray_enhanced, cv2.CV_64F).var()

    # If the model doesn't improve the sharpness, it's blurry
    return laplacian_var < threshold, laplacian_var

def extract_numbers(filename):
    """Extract the numeric parts from the filename to sort properly."""
    match = re.match(r'frame_(\d+)_(\d+)', filename)
    if match:
        return int(match.group(1)), int(match.group(2))
    return 0, 0

# Path to frames folder
frames_folder = "frames/video_2025-02-22_01-48-07/second_3"  # Change to your folder

# List to store blurry images
blurry_images = []

# Check each frame (sorted sequentially by numeric part of filename)
for filename in sorted(os.listdir(frames_folder), key=extract_numbers):
    image_path = os.path.join(frames_folder, filename)
    
    if filename.endswith(".jpg"):
        blurry, score = is_blurry(image_path)
        print(f"{filename} - Sharpness Score: {score} {'(Blurry)' if blurry else '(Clear)'}")
        
        if blurry:
            blurry_images.append(filename)  # Store blurry image names
            # os.remove(image_path)  # Delete blurry image
            print(f"Deleted: {filename}")

# Print all blurry images in sequential order
if blurry_images:
    print("\nBlurry Images Detected & Deleted:")
    for img in blurry_images:
        print(img)
else:
    print("\nNo blurry images detected.")

print("Blurry image check completed.")

frame_3_1.jpg - Sharpness Score: 7.7008025277689764 (Clear)
frame_3_2.jpg - Sharpness Score: 5.637685037931462 (Blurry)
Deleted: frame_3_2.jpg
frame_3_3.jpg - Sharpness Score: 7.246642657128349 (Clear)
frame_3_4.jpg - Sharpness Score: 7.1631282965996315 (Clear)
frame_3_5.jpg - Sharpness Score: 7.223531867739075 (Clear)
frame_3_6.jpg - Sharpness Score: 6.4660306611467915 (Blurry)
Deleted: frame_3_6.jpg
frame_3_7.jpg - Sharpness Score: 8.455189495322134 (Clear)
frame_3_8.jpg - Sharpness Score: 6.986803586437089 (Blurry)
Deleted: frame_3_8.jpg
frame_3_9.jpg - Sharpness Score: 5.371821765987438 (Blurry)
Deleted: frame_3_9.jpg
frame_3_10.jpg - Sharpness Score: 7.176176297690801 (Clear)

Blurry Images Detected & Deleted:
frame_3_2.jpg
frame_3_6.jpg
frame_3_8.jpg
frame_3_9.jpg
Blurry image check completed.


### FFT (Fast Fourier Transform) BAD

In [22]:
import cv2
import os
import re
import numpy as np

def is_blurry(image_path, threshold=100, method="laplacian"):
    """Check if an image is blurry using either Laplacian variance or FFT."""
    try:
        if method == "laplacian":
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            laplacian_var = cv2.Laplacian(image, cv2.CV_64F).var()
            return laplacian_var < threshold, laplacian_var
        elif method == "fft":
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            f = np.fft.fft2(image)
            fshift = np.fft.fftshift(f)
            magnitude_spectrum = 20 * np.log(np.abs(fshift))
            height, width = image.shape
            center_x, center_y = width // 2, height // 2
            radius = min(center_x, center_y) // 3
            mask = np.zeros_like(image, dtype=np.uint8)
            cv2.circle(mask, (center_x, center_y), radius, 255, -1)
            masked_magnitude = np.ma.masked_array(magnitude_spectrum, mask=~mask.astype(bool))
            average_high_freq = np.mean(masked_magnitude)
            return average_high_freq < 20, average_high_freq #adjust 20 as needed.
        else:
            raise ValueError("Invalid method. Choose 'laplacian' or 'fft'.")
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return True, 0  # Treat as blurry if error occurs

def extract_numbers(filename):
    """Extract the numeric parts from the filename to sort properly."""
    match = re.match(r'frame_(\d+)_(\d+)', filename)
    if match:
        return int(match.group(1)), int(match.group(2))
    return 0, 0

# Path to frames folder
frames_folder = "frames/video_2025-02-22_00-39-13"  # Change to your folder

# List to store blurry images
blurry_images = []

# Choose method 'laplacian' or 'fft'
blur_detection_method = "fft" #or "fft"

# Check each frame (sorted sequentially by numeric part of filename)
for filename in sorted(os.listdir(frames_folder), key=extract_numbers):
    image_path = os.path.join(frames_folder, filename)

    if filename.endswith(".jpg"):
        blurry, score = is_blurry(image_path, method=blur_detection_method)
        print(f"{filename} - Sharpness Score: {score} {'(Blurry)' if blurry else '(Clear)'}")

        if blurry:
            blurry_images.append(filename)
            # os.remove(image_path)  # Delete blurry image
            # print(f"Deleted: {filename}")

# Print all blurry images in sequential order
if blurry_images:
    print("\nBlurry Images Detected:")
    for img in blurry_images:
        print(img)
else:
    print("\nNo blurry images detected.")

print("Blurry image check completed.")

frame_1_1.jpg - Sharpness Score: 194.32157437249833 (Clear)
frame_1_2.jpg - Sharpness Score: 194.48981350218526 (Clear)
frame_1_3.jpg - Sharpness Score: 194.44739709781513 (Clear)
frame_1_4.jpg - Sharpness Score: 194.4096079391005 (Clear)
frame_1_5.jpg - Sharpness Score: 194.30308520340998 (Clear)
frame_1_6.jpg - Sharpness Score: 194.37084413444285 (Clear)
frame_1_7.jpg - Sharpness Score: 194.33183478449095 (Clear)
frame_1_8.jpg - Sharpness Score: 194.31660847913366 (Clear)
frame_1_9.jpg - Sharpness Score: 194.36588456970586 (Clear)
frame_1_10.jpg - Sharpness Score: 194.2782080646182 (Clear)
frame_2_1.jpg - Sharpness Score: 193.9164658439571 (Clear)
frame_2_2.jpg - Sharpness Score: 194.2718137909081 (Clear)
frame_2_3.jpg - Sharpness Score: 196.1633389425749 (Clear)
frame_2_4.jpg - Sharpness Score: 195.5412974705766 (Clear)
frame_2_5.jpg - Sharpness Score: 194.02857230346407 (Clear)
frame_2_6.jpg - Sharpness Score: 194.85505362014717 (Clear)
frame_2_7.jpg - Sharpness Score: 195.69352184

### ESPCN (Efficient Sub-Pixel Convolutional Neural Network)

###  LapSRN (Laplacian Pyramid Super-Resolution Network)